In [7]:
from sklearn.datasets import fetch_california_housing
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from TensorRing_Model import TensorRing_Based
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow import keras 
from tensorflow.keras import regularizers
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import time

def local_feature_mapping(x_train,x_val,x_test,local_dim=2,unit_norm=False):
    
    n_train_samples = x_train.shape[0]
    n_val_samples = x_val.shape[0]
    n_test_samples = x_test.shape[0]
    n_features = x_train.shape[1]

    X_transformed_train_list = []
    X_transformed_val_list = []
    X_transformed_test_list = []
    
    for d in range(local_dim-1):
        X_transformed_train_list.append(x_train**(d+1))
        X_transformed_val_list.append(x_val**(d+1))
        X_transformed_test_list.append(x_test**(d+1))
         
    X_stand_train_list = [np.ones((n_train_samples,n_features))]
    X_stand_val_list = [np.ones((n_val_samples,n_features))]
    X_stand_test_list = [np.ones((n_test_samples,n_features))]
    
    for X_trans_train,X_trans_val,X_trans_test in zip(X_transformed_train_list,X_transformed_val_list,X_transformed_test_list):
        
        # scaler=StandardScaler()
        # X_stand_train_list.append(scaler.fit_transform(X_trans_train))
        # X_stand_val_list.append(scaler.transform(X_trans_val))    
        # X_stand_test_list.append(scaler.transform(X_trans_test))  
      X_stand_train_list.append(X_trans_train)
      X_stand_val_list.append(X_trans_val) 
      X_stand_test_list.append(X_trans_test)
        
    x_train_processed= np.transpose(X_stand_train_list,(1, 2, 0))
    x_val_processed= np.transpose(X_stand_val_list,(1, 2, 0))
    x_test_processed = np.transpose(X_stand_test_list,(1, 2, 0))

    if(unit_norm==True):
      norms_train = np.sqrt(np.sum(x_train_processed**2,axis=-1,keepdims=True))
      x_train_processed = np.divide(x_train_processed,norms_train)

      norms_val = np.sqrt(np.sum(x_val_processed**2,axis=-1,keepdims=True))
      x_val_processed = np.divide(x_val_processed,norms_val)
    
      norms_test = np.sqrt(np.sum(x_test_processed**2,axis=-1,keepdims=True))
      x_test_processed = np.divide(x_test_processed,norms_test)

    return x_train_processed, x_val_processed, x_test_processed      

    ### Data preprocessing ###
dataset = fetch_california_housing()
X_full, Y_full = dataset.data, dataset.target
x_train, x_test, y_train, y_test = train_test_split(X_full, Y_full, test_size = 0.2, random_state = 0)
x_train, x_val, y_train, y_val = x_train[:13208],x_train[13208:], y_train[:13208],y_train[13208:]

scaler_X = StandardScaler()
scaler_Y = StandardScaler()

x_train, x_val, x_test= scaler_X.fit_transform(x_train), scaler_X.transform(x_val), scaler_X.transform(x_test)
y_train, y_val, y_test = scaler_Y.fit_transform(y_train.reshape(-1,1)).ravel(), scaler_Y.transform(y_val.reshape(-1,1)).ravel(), scaler_Y.transform(y_test.reshape(-1,1)).ravel()

#%%
rank=5
local_dim=75

x_train_processed,x_val_processed,x_test_processed=local_feature_mapping(x_train,x_val,x_test,local_dim=local_dim,unit_norm=True)

print('rank: {}, local dimension:{}'.format(rank,local_dim))

model=Sequential()
model.add(TensorRing_Based(units=1,activation=None,rank=rank,local_dim=local_dim,regularizer=keras.regularizers.l2(3e-7),

                    initializer=keras.initializers.TruncatedNormal(mean=0, stddev=0.2)))

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())

start = time.time()
history=model.fit(x_train_processed, y_train, epochs=100, batch_size=32,validation_data=(x_val_processed,y_val))
end = time.time()

val_loss=min(history.history['val_loss'])
print('validation loss:{}'.format(val_loss))
   
print('Training time:{}'.format(end-start))


rank: 5, local dimension:75
Train on 13208 samples, validate on 3304 samples
Epoch 1/100
13208/13208 [==============================] - 5s 346us/sample - loss: 0.6351 - val_loss: 0.3890
Epoch 2/100
13208/13208 [==============================] - 3s 208us/sample - loss: 0.3540 - val_loss: 0.3008
Epoch 3/100
13208/13208 [==============================] - 3s 201us/sample - loss: 0.2842 - val_loss: 0.2604
Epoch 4/100
13208/13208 [==============================] - 2s 172us/sample - loss: 0.2517 - val_loss: 0.2425
Epoch 5/100
13208/13208 [==============================] - 2s 172us/sample - loss: 0.2331 - val_loss: 0.2298
Epoch 6/100
13208/13208 [==============================] - 2s 173us/sample - loss: 0.2205 - val_loss: 0.2209
Epoch 7/100
13208/13208 [==============================] - 2s 173us/sample - loss: 0.2136 - val_loss: 0.2226
Epoch 8/100
13208/13208 [==============================] - 3s 195us/sample - loss: 0.2061 - val_loss: 0.2154
Epoch 9/100
13208/13208 [==========================

Epoch 75/100
13208/13208 [==============================] - 2s 179us/sample - loss: 0.1486 - val_loss: 0.2179
Epoch 76/100
13208/13208 [==============================] - 3s 198us/sample - loss: 0.1495 - val_loss: 0.2247
Epoch 77/100
13208/13208 [==============================] - 2s 186us/sample - loss: 0.1493 - val_loss: 0.2208
Epoch 78/100
13208/13208 [==============================] - 3s 239us/sample - loss: 0.1484 - val_loss: 0.2256
Epoch 79/100
13208/13208 [==============================] - 2s 179us/sample - loss: 0.1482 - val_loss: 0.2216
Epoch 80/100
13208/13208 [==============================] - 3s 211us/sample - loss: 0.1480 - val_loss: 0.2218
Epoch 81/100
13208/13208 [==============================] - 3s 198us/sample - loss: 0.1479 - val_loss: 0.2204
Epoch 82/100
13208/13208 [==============================] - 3s 202us/sample - loss: 0.1476 - val_loss: 0.2194
Epoch 83/100
13208/13208 [==============================] - 2s 179us/sample - loss: 0.1460 - val_loss: 0.2297
Epoch 84/1